<a href="https://colab.research.google.com/github/tharushaliyanagama/OralCancerEarlyDetection-DSGP/blob/Image-Prediction/ResNet50_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
import os
import random
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Paths to your dataset
data_dir = "/content/drive/MyDrive/Proper_dataset"
train_dir = os.path.join(data_dir, "train")  # Replace with your dataset's train folder
val_dir = os.path.join(data_dir, "val")      # Replace with your dataset's validation folder

In [4]:
# Parameters
batch_size = 32
img_height = 224
img_width = 224

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2603 images belonging to 2 classes.
Found 651 images belonging to 2 classes.


In [5]:
def create_resnet50_model():
    base_model = tf.keras.applications.ResNet50(input_shape=(img_height, img_width, 3),
                                                include_top=False,
                                                weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [6]:
def generate_classification_report(model, val_data):
    # Get predictions from the validation data
    val_data.reset()  # Reset the generator to avoid skipping images
    y_true = val_data.classes
    y_pred = model.predict(val_data)
    y_pred = np.round(y_pred).astype(int)  # Convert probabilities to 0 or 1

    # Generate and print classification report
    report = classification_report(y_true, y_pred, target_names=val_data.class_indices.keys())
    print(report)

In [7]:
resnet50_model = create_resnet50_model()

print("ResNet50 Model Summary:")
resnet50_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
ResNet50 Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train ResNet50 model
print("Training ResNet50 model...")
resnet50_history = resnet50_model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


Training ResNet50 model...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.8564 - loss: 0.4256

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


82/82 ━━━━━━━━━━━━━━━━━━━━ 1005s 11s/step - accuracy: 0.8563 - loss: 0.4257 - val_accuracy: 0.8479 - val_loss: 0.4210
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 575s 7s/step - accuracy: 0.8541 - loss: 0.4123 - val_accuracy: 0.8479 - val_loss: 0.4205
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 577s 7s/step - accuracy: 0.8498 - loss: 0.4179 - val_accuracy: 0.8479 - val_loss: 0.4116
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 578s 7s/step - accuracy: 0.8528 - loss: 0.4013 - val_accuracy: 0.8479 - val_loss: 0.3986
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 580s 7s/step - accuracy: 0.8417 - loss: 0.4167 - val_accuracy: 0.8479 - val_loss: 0.3923
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 579s 7s/step - accuracy: 0.8441 - loss: 0.4119 - val_accuracy: 0.8479 - val_loss: 0.3974
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 622s 7s/step - accuracy: 0.8398 - loss: 0.4139 - val_accuracy: 0.8479 - val_loss: 0.3814
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 654s 7s/step - accuracy: 0.8516 - loss: 0.3841 - val_accuracy: 0.8479 - val_loss: 